# BEM114 Homework 4 - Risk-Weighted Portfolios
**Names:** Andrew Zabelo, Daniel Wen, Kyle McCandless  
**Student IDs:** 2176083, 2159859, 2157818

## Setup

### Imports and Helper Functions

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [2]:
# Calculates returns and prints the returns mean, vol, and Sharpe ratio for a strategy
def analyze(returns, strat_name):
    strat_mean = returns.mean()
    strat_vol = returns.std()
    strat_sharpe = strat_mean / strat_vol
    print(f"{strat_name} monthly returns:")
    print(f"Mean = {strat_mean*100:.3f}%")
    print(f"Volatility = {strat_vol:0.3f}%")
    print(f"Sharpe Ratio = {strat_sharpe:0.3f}")

# Estimates the CAPM, FF3 and FF5+MOM models on df using the returns found in ret_col_name
def estimate_models(df, ret_col_name):
    # Estimate CAPM
    print('CAPM')
    print(sm.OLS(df[ret_col_name], sm.add_constant(df[['Mkt-RF']])).fit().summary())
    
    # Estimate FF3
    print('FF3')
    print(sm.OLS(df[ret_col_name], sm.add_constant(df[['Mkt-RF', 'SMB', 'HML']])).fit().summary())
    
    # Estimate FF5
    print('FF5 + MOM')
    print(sm.OLS(df[ret_col_name], sm.add_constant(df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']])).fit().summary())

### Load monthly and daily FF5 and MOM returns data

In [3]:
mff5 = pd.read_csv('data/FF5_monthly.csv')
mff5['date'] = pd.to_datetime(mff5['date'], format='%Y%m')

dff5 = pd.read_csv('data/FF5_daily.csv')
dff5['date'] = pd.to_datetime(dff5['date'], format='%Y%m%d')

mmom = pd.read_csv('data/MOM_monthly.csv')
mmom['date'] = pd.to_datetime(mmom['date'], format='%Y%m')

dmom = pd.read_csv('data/MOM_daily.csv')
dmom['date'] = pd.to_datetime(dmom['date'], format='%Y%m%d')

## Problem 1

### Part A

In [4]:
# Merge monthly FF5 and MOM data
mdf = pd.merge(mff5, mmom, on='date', how='outer')
assert(len(mdf) == len(mmom))

# Set dates to months with no days
mdf['date'] = mdf['date'].dt.to_period('M')
mdf

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM
0,1963-07,-0.39,-0.41,-0.97,0.68,-1.18,0.27,0.90
1,1963-08,5.07,-0.80,1.80,0.36,-0.35,0.25,1.01
2,1963-09,-1.57,-0.52,0.13,-0.71,0.29,0.27,0.19
3,1963-10,2.53,-1.39,-0.10,2.80,-2.01,0.29,3.12
4,1963-11,-0.85,-0.88,1.75,-0.51,2.24,0.27,-0.74
...,...,...,...,...,...,...,...,...
1161,1963-02,NaN,NaN,NaN,NaN,NaN,NaN,2.53
1162,1963-03,NaN,NaN,NaN,NaN,NaN,NaN,1.62
1163,1963-04,NaN,NaN,NaN,NaN,NaN,NaN,-0.09
1164,1963-05,NaN,NaN,NaN,NaN,NaN,NaN,0.33


In [5]:
# Merge daily FF5 and MOM data
ddf = pd.merge(dff5, dmom, on='date', how='outer')
assert(len(ddf) == len(dmom))
ddf

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012,-0.21
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012,0.42
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012,0.41
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012,0.07
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012,-0.45
...,...,...,...,...,...,...,...,...
25604,1963-06-24,NaN,NaN,NaN,NaN,NaN,NaN,0.43
25605,1963-06-25,NaN,NaN,NaN,NaN,NaN,NaN,0.00
25606,1963-06-26,NaN,NaN,NaN,NaN,NaN,NaN,-0.42
25607,1963-06-27,NaN,NaN,NaN,NaN,NaN,NaN,-0.34


### Part B

In [6]:
FACTORS = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']
FACTORS_var = [name + '_var' for name in FACTORS]
FACTORS_lag = [name + '_lag' for name in FACTORS]

# Compute 22-day rolling variance
ddf[FACTORS_var] = ddf[FACTORS].rolling(window=22, min_periods=22).var()
ddf

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM,Mkt-RF_var,SMB_var,HML_var,RMW_var,CMA_var,MOM_var
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012,-0.21,NaN,NaN,NaN,NaN,NaN,NaN
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012,0.42,NaN,NaN,NaN,NaN,NaN,NaN
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012,0.41,NaN,NaN,NaN,NaN,NaN,NaN
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012,0.07,NaN,NaN,NaN,NaN,NaN,NaN
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012,-0.45,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25604,1963-06-24,NaN,NaN,NaN,NaN,NaN,NaN,0.43,NaN,NaN,NaN,NaN,NaN,0.040409
25605,1963-06-25,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.038302
25606,1963-06-26,NaN,NaN,NaN,NaN,NaN,NaN,-0.42,NaN,NaN,NaN,NaN,NaN,0.049502
25607,1963-06-27,NaN,NaN,NaN,NaN,NaN,NaN,-0.34,NaN,NaN,NaN,NaN,NaN,0.054316


### Part C

In [7]:
# Collapse data to end of month only
ddf = ddf[ddf['date'].dt.is_month_end]

ddf[FACTORS_lag] = ddf[FACTORS_var].shift(1)

for f in FACTORS:
    # Drop NaNs and complete regression of variance on last month's variance
    temp = ddf.dropna(subset=[f+'_lag', f+'_var'])
    print(f'\n\n{f}:')
    print(sm.OLS(temp[f'{f}_var'], sm.add_constant(temp[f'{f}_lag'])).fit().summary())



Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:             Mkt-RF_var   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     95.93
Date:                Tue, 07 May 2024   Prob (F-statistic):           7.39e-21
Time:                        15:01:46   Log-Likelihood:                -1076.9
No. Observations:                 508   AIC:                             2158.
Df Residuals:                     506   BIC:                             2166.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6281      0.099      6.33

For all 5 Fama French factors and momentum, the coefficient on lagged variance is positive and significant at the 1% level, indicating that the variance is time-persistent. Variance in the last 22 days of a previous month positively correlates with variance in the last 22 days of the next month.

### Part D

In [8]:
for f in FACTORS:
    # Drop NaNs and complete regression of returns on last month's variance
    temp = ddf.dropna(subset=[f, f+'_lag'])
    print(f'\n\n{f}:')
    print(sm.OLS(temp[f], sm.add_constant(temp[f'{f}_lag'])).fit().summary())



Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                   0.05919
Date:                Tue, 07 May 2024   Prob (F-statistic):              0.808
Time:                        15:01:46   Log-Likelihood:                -695.36
No. Observations:                 508   AIC:                             1395.
Df Residuals:                     506   BIC:                             1403.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0713      0.047      1.52

The Fama French 5 factors' coefficients all fail to be significant at the 5% level, indicating that variance doesn't strongly predict returns for these factors. However, the coefficient for momentum is negative and significant at the 5% level, indicating that variance does negatively predict returns for the momentum factor.

### Part E

In [9]:
'''
Calculate volatility managed portfolio weights
'''

WEIGHTS = ['w_Mkt-RF', 'w_SMB', 'w_HML', 'w_RMW', 'w_CMA', 'w_MOM']

p = ddf.drop(FACTORS_lag, axis=1)
for f in FACTORS:
    # Create each factor's weight
    p['w_'+f] = p[f+'_var'].mean() / p[f+'_var']

p = p.drop(FACTORS_var, axis=1)
p

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM,w_Mkt-RF,w_SMB,w_HML,w_RMW,w_CMA,w_MOM
21,1963-07-31,-0.13,0.11,-0.03,-0.13,0.30,0.012,0.01,4.661864,13.657852,8.891070,7.175286,4.354141,7.481772
63,1963-09-30,-0.60,0.21,0.08,0.24,0.13,0.014,-0.56,6.762804,5.439042,9.608496,5.532341,4.907465,7.931046
86,1963-10-31,0.21,-0.03,0.08,0.10,-0.26,0.013,0.11,6.086183,3.691715,2.892235,2.577082,1.607613,6.106420
125,1963-12-31,0.56,0.44,-0.09,0.17,-0.22,0.014,-0.14,6.088929,3.300998,7.835700,2.493463,1.619211,6.675612
147,1964-01-31,0.38,-0.33,-0.17,0.35,-0.28,0.013,0.00,15.877211,4.009739,2.797540,1.424019,1.275599,8.906166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,1962-12-31,NaN,NaN,NaN,NaN,NaN,NaN,0.19,NaN,NaN,NaN,NaN,NaN,2.624661
25505,1963-01-31,NaN,NaN,NaN,NaN,NaN,NaN,-0.17,NaN,NaN,NaN,NaN,NaN,2.047161
25524,1963-02-28,NaN,NaN,NaN,NaN,NaN,NaN,0.29,NaN,NaN,NaN,NaN,NaN,17.232779
25566,1963-04-30,NaN,NaN,NaN,NaN,NaN,NaN,0.32,NaN,NaN,NaN,NaN,NaN,5.324392


In [10]:
'''
Calculate volatility managed portfolio returns
'''

# Drop daily returns and pull in monthly returns instead
p = p.drop(FACTORS+['RF'], axis=1)

# Merge with purely monthly data
p['date'] = p['date'].dt.to_period('M')
p = pd.merge(p, mdf, on=['date'], how='inner')

# Shift all weights and drop the resulting NaNs
p[WEIGHTS] = p[WEIGHTS].shift(1)
p = p.dropna(subset=WEIGHTS+FACTORS+['RF'])

# Calculate returns -- leave out RF returns according to the hint
for name in FACTORS:
    p[name+"_ret"] = p[name]*p['w_'+name] #+ p['RF']*(1 - p['w_'+name])

p = p.drop(WEIGHTS, axis=1)
p

,date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM,Mkt-RF_ret,SMB_ret,HML_ret,RMW_ret,CMA_ret,MOM_ret
1,1963-09,-1.57,-0.52,0.13,-0.71,0.29,0.27,0.19,-7.319127,-7.102083,1.155839,-5.094453,1.262701,1.421537
2,1963-10,2.53,-1.39,-0.10,2.80,-2.01,0.29,3.12,17.109894,-7.560268,-0.960850,15.490555,-9.864005,24.744865
3,1963-12,1.83,-2.10,-0.02,0.03,-0.07,0.29,1.75,11.137715,-7.752601,-0.057845,0.077312,-0.112533,10.686234
4,1964-01,2.24,0.13,1.48,0.17,1.47,0.30,0.86,13.639200,0.429130,11.596835,0.423889,2.380240,5.741026
5,1964-03,1.41,1.23,3.40,-2.21,3.22,0.31,0.75,22.386867,4.931979,9.511637,-3.147081,4.107429,6.679625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2023-08,-2.39,-3.65,-1.06,3.43,-2.37,0.45,3.77,-7.210065,-2.245514,-0.602433,1.953692,-1.410968,3.237485
505,2023-10,-3.19,-4.05,0.19,2.46,-0.66,0.47,1.73,-5.067056,-8.297694,0.207198,2.800363,-0.711740,3.551963
506,2023-11,8.84,-0.12,1.64,-3.91,-1.00,0.44,2.75,10.674656,-0.083116,1.518157,-1.452097,-0.944850,2.822849
507,2024-01,0.71,-5.74,-2.38,0.69,-0.96,0.47,5.18,1.351876,-1.426036,-1.952367,0.310427,-1.458591,2.804609


In [11]:
for name in FACTORS:
    analyze(p[name+'_ret'], f'{name} volatility-managed portfolio')
    print()

Mkt-RF volatility-managed portfolio monthly returns:
Mean = 89.068%
Volatility = 12.671%
Sharpe Ratio = 0.070

SMB volatility-managed portfolio monthly returns:
Mean = 10.636%
Volatility = 8.791%
Sharpe Ratio = 0.012

HML volatility-managed portfolio monthly returns:
Mean = 89.924%
Volatility = 9.269%
Sharpe Ratio = 0.097

RMW volatility-managed portfolio monthly returns:
Mean = 82.025%
Volatility = 5.165%
Sharpe Ratio = 0.159

CMA volatility-managed portfolio monthly returns:
Mean = 43.067%
Volatility = 4.096%
Sharpe Ratio = 0.105

MOM volatility-managed portfolio monthly returns:
Mean = 399.095%
Volatility = 15.504%
Sharpe Ratio = 0.257



### Part F

In [12]:
for name in FACTORS:
    print('----------------------------------------------------------------------------')
    print(name)
    print('----------------------------------------------------------------------------')
    estimate_models(p, name+'_ret')

----------------------------------------------------------------------------
Mkt-RF
----------------------------------------------------------------------------
CAPM
                            OLS Regression Results                            
Dep. Variable:             Mkt-RF_ret   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     339.4
Date:                Tue, 07 May 2024   Prob (F-statistic):           2.24e-58
Time:                        15:01:46   Log-Likelihood:                -1879.9
No. Observations:                 508   AIC:                             3764.
Df Residuals:                     506   BIC:                             3772.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t 

RMW, CMA, MOM at 1% under all models. HML is under CAPM, FF5+MOM but not FF3

The RMW strategy produces positive and significant alpha at the 1% level under the CAPM, FF3, and FF5+MOM models. The CMA strategy produces positive and significant alpha at the 5% level under FF5+MOM, but not under the other two models. Finally, the SMB strategy produces positive and significant alpha at the 1% level under CAPM, but not under the other two models.

As we found in Part C, the variance of each of these factors positively predicts future variance. Our strategy gives low weights to currently high-variance factors, and higher weights when the factors are lower variance. As we discussed in class, this volatility managed portfolio is taking advantage of the discovery that high volatility assets tend to underperform. Because the current variance of these factors predicts the future variance, underweighting them in times of high variance at one month backs the strategy out of the next month that is statistically likely to see underperforming returns for that risk level.

## Problem 2

### Part A